### Scrape výberových konaní

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

import pickle
import sys
import time

In [2]:
datestring = '20200210'

In [3]:
# nacitaj relativne linky ku konaniam
with open('./Data/indexy_konani_list_'+datestring+'.pkl', 'rb') as f:
    indexy_konani = pickle.load(f)

In [4]:
# definuj url
url_detail = 'https://open.slovensko.sk/VK/SelectionProcedureDetail/'
url_frame = 'https://open.slovensko.sk/VK/GetSelectionProcedureDetailFrame/'
url0 = 'https://open.slovensko.sk/VK/SelectionProcedureListSearch'

In [5]:
# data ktore ulozime z SelectionProcedureDetailFrame
vars_dict = {} # label:var
vars_dict['Identifikátor výberového konania'] = 'vk_id'
vars_dict['Služobný úrad'] = 'sluzobny_urad'
vars_dict['Sídlo služobného úradu'] = 'sluzobny_urad_sidlo'
vars_dict['Druh výberoveho konania'] = 'vk_druh'
vars_dict['Počet obsadzovaných miest'] = 'pocet_obsadzovane'
vars_dict['Organizačný útvar'] = 'org_utvar'
vars_dict['Nadriadený organizačný útvar'] = 'nad_org_utvar'
vars_dict['Odbor štátnej služby'] = 'statna_sluzba_odbor'
vars_dict['Obsadzované štátnozamestnanecké miesto vo funkcii'] = 'funkcia'
vars_dict['Druh štátnej služby'] = 'statna_sluzba_druh'
vars_dict['Pravidelné miesto výkonu štátnej služby'] = 'miesto_vykonu'
vars_dict['Vzdelanie'] = 'vzdelanie'
vars_dict['Dátum vyhlásenia výberového konania'] = 'datum_vyhlasenia'
vars_dict['Termín na podanie žiadosti o zaradenie do výberového konania spolu s ďalšími požadovanými dokumentmi'] = 'datum_podania'
#
vars_dict['Vedúci štátny zamestnanec'] = 'veduci'
vars_dict['Najnáročnejšia činnosť'] = 'najnarocnejsia_cinnost'
vars_dict['Bližšie určená najnáročnejšia činnosť'] = 'najnarocnejsia_cinnost_blizsie'
vars_dict['Vyžaduje sa ovládanie cudzieho jazyka'] = 'cudzi_jazyk'
vars_dict['Vyžaduje sa odborná prax'] = 'odborna_prax'
vars_dict['Vyžaduje sa oprávnenie na oboznamovanie sa s utajovanými skutočnosťami'] = 'utajovane_skutocnosti'
vars_dict['Ďalšie požiadavky'] = 'dalsie_poziadavky'

# data, ktore vytiahneme z SelectionProcedureDetail
stlpce = ['index_konania', 'konanie_uspesne', 'pocet_vybrani', 'pocet_dalsi', 'pocet_neuspesni', 'pocet_prihlaseni']
# vsetky stahovane data
stlpce = stlpce + list(vars_dict.values())

# pre niektore upravme datove typy
stlpce_int = ['index_konania', 'pocet_vybrani', 'pocet_dalsi', 'pocet_neuspesni', 'pocet_prihlaseni', 'pocet_obsadzovane']
stlpce_bool = ['konanie_uspesne'] 
stlpce_datum = ['datum_vyhlasenia', 'datum_podania']
stlpce_string = list(filter(lambda x: x not in stlpce_int+stlpce_bool+stlpce_datum, stlpce))

In [6]:
# vytvorme dataframe
df = pd.DataFrame(columns=stlpce)

# nastavme dtype stlpcov
df[stlpce_int] = df[stlpce_int].astype(int)
df[stlpce_bool] = df[stlpce_bool].astype(bool)
df[stlpce_datum] = df[stlpce_datum].astype('datetime64[ns]')
df[stlpce_string] = df[stlpce_string].astype(str)

In [7]:
# nepritomne k 20191118
df_preskocene = pd.DataFrame(columns=['vysledok'])

In [8]:
driver = webdriver.Firefox()
driver.get(url0)

In [9]:
driver_reset_counter = 0
file_counter = 0
# driver_reset_counter = 16901
for index_konania in indexy_konani:   
    # resetuj driver raz za cas
    # dirty riesenie na zapratavanie pamate
    if driver_reset_counter != 0 and driver_reset_counter % 100 == 0:
        # print cas
        file_counter += 1
        print(driver_reset_counter//100, file_counter, time.localtime()[3], ':', time.localtime()[4])
        # zavri prehliadac
        driver.close()
        time.sleep(2)
        
        # uloz data
        with open('./Data/data_'+datestring+'_'+str(file_counter)+'.pkl', 'wb') as f:
            pickle.dump(df, f)
    
        #reset data frame
        df = pd.DataFrame(columns=stlpce)

        df[stlpce_int] = df[stlpce_int].astype(int)
        df[stlpce_string] = df[stlpce_string].astype(str)

        #reset driver
        driver = webdriver.Firefox()
        driver.get(url0)
        time.sleep(1)
    
    # nacitaj stranku s poctami a vysledkom
    url = url_detail + index_konania
    driver.get(url)
    time.sleep(0.1)
    soup = BeautifulSoup(driver.page_source)
    
    #
    # vysledok konania
    #

    vysledok = None
    for elem in soup.findAll("h2"):
        # najdi vysledok
        if 'výsledok' in elem.getText():
            vysledok = elem.getText()

    # vysledok je posledny z h2.
    if vysledok is None:
        # log VK ine ako tie s vyhlasenym vysledkom
        print('Preskocene i', index_konania, 'Text:', elem.getText())
        with open('./Data/log_preskocene_'+datestring+'.txt', 'a') as f:
            log_str = 'Preskocene ' + str(index_konania) + ' Text: ' + elem.getText() + '\n'
            f.write(log_str)
            df_preskocene.loc[index_konania, 'vysledok'] = elem.getText()
        continue # neukladaj
    else:
        if 'neúspešné' in vysledok:
            konanie_uspesne = False
        else:
            konanie_uspesne = True

    #
    # pocty uchadzacov
    #
    
    pocet_vybrani = 0
    pocet_dalsi = 0
    pocet_neuspesni = 0
    pocet_prihlaseni = 0
    for elem in soup.findAll("div", class_="subpagePnl"):
        popisok = elem.h4.getText()
        if 'Vybraní' in popisok or 'Úspešní uchádzači' in popisok:
            pocet_vybrani = len(elem.findAll('strong'))
        elif 'Ďalší' in popisok:
            pocet_dalsi = len(elem.findAll('strong'))
        elif 'Neúspešní' in popisok:
            pocet_neuspesni = len(elem.findAll('strong'))
        elif 'Počet' in popisok:
            pocet_prihlaseni = int(elem.p.getText().replace(' ', '').replace('\n', ''))
        else:
            raise Exception(popisok)
            
    # uloz data ziskane doteraz
    konanie_dict = {}
    konanie_dict['index_konania'] = index_konania
    konanie_dict['konanie_uspesne'] = konanie_uspesne
    konanie_dict['pocet_vybrani'] = pocet_vybrani
    konanie_dict['pocet_dalsi'] = pocet_dalsi
    konanie_dict['pocet_neuspesni'] = pocet_neuspesni
    konanie_dict['pocet_prihlaseni'] = pocet_prihlaseni
    
    # nacitaj stranku s informaciami SelectionProcedureDetailFrame
    url = url_frame + index_konania
    driver.get(url)
    time.sleep(0.1)
    soup = BeautifulSoup(driver.page_source)
    
    # nacitaj popisky
    # musi byt trochu vseobecne, nie vsetky polia su zhodne medzi konaniami
    labels = list(map(lambda x: str(x.getText()), soup.findAll('span', class_='fieldLabel')))
    # najdi datum
    datum_index = labels.index('Dátum vyhlásenia výberového konania')
    # nacitaj hodnoty
    vals = soup.findAll('div', class_="contentVis wordwrap")
    # pred datumom sa nachadzaju pozadovane dokumenty a forma podania ziadosti - nezaujimave a nezapada do struktury
    # vyhodime
    vals.pop(datum_index)
    vals.pop(datum_index)
    # linebreaky nahradim newline, nahradime nezlomitelnu medzeru \xa0 obycajnou medzerou
    vals = list(map(lambda x: str(x.getText("\n")).replace('\xa0', ' '), vals))
    
    # najdi a uloz data
    for field_label in vars_dict.keys():
        premenna = vars_dict[field_label]
        try:
            hodnota = vals[labels.index(field_label)]
        except ValueError:
            hodnota = ''

        if premenna in stlpce_string:
            konanie_dict[premenna] = hodnota
        if premenna in stlpce_int:
            if hodnota == '':
                hodnota = -1
            konanie_dict[premenna] = int(hodnota)
        elif premenna in stlpce_datum:
            datum_list = hodnota.split('.')
            datum_list = list(map(int, datum_list))
            konanie_dict[premenna] = pd.Timestamp(day=datum_list[0], month=datum_list[1], year=datum_list[2])

    df = df.append(konanie_dict, ignore_index=True)
    driver_reset_counter += 1

Preskocene i 13876 Text: Zrušené výberové konanie
Preskocene i 3312 Text: Ukončené prihlasovanie
Preskocene i 24488 Text: Ukončené prihlasovanie
Preskocene i 23077 Text: Ukončené prihlasovanie
Preskocene i 5703 Text: Zrušené výberové konanie
Preskocene i 8094 Text: Zrušené výberové konanie
Preskocene i 12236 Text: Zrušené výberové konanie
Preskocene i 19584 Text: Zrušené výberové konanie
Preskocene i 19561 Text: Zrušené výberové konanie
Preskocene i 24678 Text: Vyhlásené
Preskocene i 22294 Text: Ukončené prihlasovanie
Preskocene i 21958 Text: Zrušené výberové konanie
Preskocene i 951 Text: Zrušené výberové konanie
Preskocene i 24637 Text: Vyhlásené
Preskocene i 24363 Text: Ukončené prihlasovanie
Preskocene i 22563 Text: Ukončené prihlasovanie
Preskocene i 24526 Text: Ukončené prihlasovanie
Preskocene i 4273 Text: Zrušené výberové konanie
Preskocene i 14402 Text: Zrušené výberové konanie
Preskocene i 24387 Text: Ukončené prihlasovanie
Preskocene i 2840 Text: Zrušené výberové konanie
Pre

Preskocene i 15380 Text: Zrušené štátnozamestnanecké miesto
Preskocene i 5866 Text: Zrušené výberové konanie
Preskocene i 3487 Text: Ukončené prihlasovanie
Preskocene i 2539 Text: Zrušené výberové konanie
Preskocene i 24384 Text: Ukončené prihlasovanie
Preskocene i 19379 Text: Zrušené výberové konanie
Preskocene i 24580 Text: Vyhlásené
Preskocene i 12500 Text: Zrušené výberové konanie
Preskocene i 5399 Text: Ukončené prihlasovanie
Preskocene i 22601 Text: Zrušené výberové konanie
Preskocene i 24680 Text: Vyhlásené
Preskocene i 6866 Text: Zánik dôvodu na obsadenie štátnozamestnaneckého miesta
Preskocene i 10666 Text: Zrušené výberové konanie
Preskocene i 24243 Text: Ukončené prihlasovanie
Preskocene i 9500 Text: Zrušené výberové konanie
Preskocene i 22196 Text: Ukončené prihlasovanie
Preskocene i 3043 Text: Zrušené výberové konanie
Preskocene i 24365 Text: Ukončené prihlasovanie
Preskocene i 16577 Text: Zrušené výberové konanie
Preskocene i 13676 Text: Zrušené výberové konanie
Preskocen

Preskocene i 2023 Text: Zrušené výberové konanie
Preskocene i 13984 Text: Zrušené výberové konanie
Preskocene i 5851 Text: Zrušené výberové konanie
Preskocene i 14643 Text: Zánik dôvodu na obsadenie štátnozamestnaneckého miesta
Preskocene i 11143 Text: Zrušené výberové konanie
Preskocene i 24618 Text: Vyhlásené
Preskocene i 22865 Text: Ukončené prihlasovanie
Preskocene i 22241 Text: Ukončené prihlasovanie
Preskocene i 2409 Text: Zrušené výberové konanie
Preskocene i 13956 Text: Zrušené výberové konanie
Preskocene i 1381 Text: Zrušené výberové konanie
Preskocene i 24541 Text: Vyhlásené
Preskocene i 20065 Text: Ukončené prihlasovanie
Preskocene i 10616 Text: Ukončené prihlasovanie
Preskocene i 6154 Text: Zrušené výberové konanie
Preskocene i 10986 Text: Zrušené výberové konanie
Preskocene i 24257 Text: Ukončené prihlasovanie
Preskocene i 24656 Text: Vyhlásené
Preskocene i 10811 Text: Zrušené výberové konanie
Preskocene i 22182 Text: Zrušené výberové konanie
Preskocene i 24593 Text: Vyhlá

Preskocene i 20434 Text: Ukončené prihlasovanie
Preskocene i 21872 Text: Ukončené prihlasovanie
Preskocene i 16322 Text: Zrušené výberové konanie
Preskocene i 715 Text: Zrušené výberové konanie
Preskocene i 24583 Text: Vyhlásené
Preskocene i 19586 Text: Zrušené výberové konanie
Preskocene i 15585 Text: Ukončené prihlasovanie
8 10 13 : 38
Preskocene i 24357 Text: Ukončené prihlasovanie
8 11 13 : 39
Preskocene i 24645 Text: Vyhlásené
Preskocene i 24501 Text: Ukončené prihlasovanie
Preskocene i 24228 Text: Ukončené prihlasovanie
Preskocene i 19660 Text: Zrušené výberové konanie
Preskocene i 24481 Text: Zrušené výberové konanie
Preskocene i 24610 Text: Vyhlásené
Preskocene i 1103 Text: Zrušené štátnozamestnanecké miesto
Preskocene i 17369 Text: Zrušené výberové konanie
Preskocene i 19552 Text: Zrušené výberové konanie
Preskocene i 24552 Text: Vyhlásené
Preskocene i 3532 Text: Ukončené prihlasovanie
Preskocene i 15335 Text: Ukončené prihlasovanie
Preskocene i 1384 Text: Zrušené výberové kon

Preskocene i 1884 Text: Zrušené výberové konanie
Preskocene i 24476 Text: Ukončené prihlasovanie
Preskocene i 9683 Text: Ukončené prihlasovanie
Preskocene i 12855 Text: Zrušené výberové konanie
Preskocene i 18114 Text: Zrušené výberové konanie
Preskocene i 12552 Text: Zrušené výberové konanie
Preskocene i 24574 Text: Ukončené prihlasovanie
Preskocene i 18707 Text: Ukončené prihlasovanie
Preskocene i 20291 Text: Ukončené prihlasovanie
Preskocene i 758 Text: Zrušené výberové konanie
Preskocene i 24410 Text: Ukončené prihlasovanie
Preskocene i 24191 Text: Zrušené výberové konanie
Preskocene i 17276 Text: Zrušené výberové konanie
Preskocene i 2030 Text: Zrušené výberové konanie
Preskocene i 9585 Text: Zrušené výberové konanie
Preskocene i 2372 Text: Zrušené výberové konanie
Preskocene i 12106 Text: Zrušené výberové konanie
Preskocene i 12331 Text: Zrušené výberové konanie
Preskocene i 14051 Text: Zrušené výberové konanie
Preskocene i 19643 Text: Zrušené výberové konanie
11 14 13 : 45
Presk

Preskocene i 484 Text: Zrušené výberové konanie
Preskocene i 24346 Text: Ukončené prihlasovanie
Preskocene i 18750 Text: Ukončené prihlasovanie
Preskocene i 8283 Text: Ukončené prihlasovanie
Preskocene i 9982 Text: Ukončené prihlasovanie
Preskocene i 24591 Text: Vyhlásené
Preskocene i 22331 Text: Ukončené prihlasovanie
Preskocene i 24517 Text: Ukončené prihlasovanie
Preskocene i 24639 Text: Vyhlásené
Preskocene i 14016 Text: Zrušené výberové konanie
Preskocene i 19578 Text: Zrušené výberové konanie
Preskocene i 24644 Text: Vyhlásené
Preskocene i 12655 Text: Zrušené výberové konanie
Preskocene i 24395 Text: Ukončené prihlasovanie
Preskocene i 24590 Text: Vyhlásené
Preskocene i 14437 Text: Zrušené výberové konanie
Preskocene i 22861 Text: Ukončené prihlasovanie
Preskocene i 18864 Text: Zrušené výberové konanie
Preskocene i 24556 Text: Ukončené prihlasovanie
Preskocene i 11133 Text: Zrušené výberové konanie
Preskocene i 22674 Text: Zánik dôvodu na obsadenie štátnozamestnaneckého miesta
Pr

In [11]:
# uloz posledny df
with open('./Data/data_'+datestring+'_'+str(file_counter + 1)+'.pkl', 'wb') as f:
            pickle.dump(df, f)

In [14]:
with open('./Data/df_preskocene_'+datestring+'.pkl', 'wb') as f:
    pickle.dump(df_preskocene, f)